In [2]:
import pandas as pd
import numpy as np
import string

# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [3]:
dataset = pd.read_csv(r'train_data.csv')
dataset['text'].values[1]

'\ufeff                       日本 去年 海外 资产 和 负债额 均 创 历史纪录新华社 东京 ５ 月 ２ ５ 日电 日本 内阁 今天 发表 的 一份 报告书 表明 ， 去年 日本 海外 资产额 和 外债 额均 创 历史 最高 纪录 。去年 日本 海外 资产额 已达 １ · ７ ７ １ 万亿美元 ， 比 １９ ８ ８ 年 增加 百分之二十点 五 。 海外 资产 中 增加 最快 的 是民间 部门 的 直接 投资 。 在 制造业 方面 ， 日本 由于 大量 购买外国 的 不动产 和 在外 办厂 等 ， 资产额 比 １ ９ ８ ８ 年 增加 百分 之 三十九点 三 ， 达 １ ５ ４ ３ 亿美元 。去年 日本 对外 负债额 为 １ · ４ ７ ７ ７ 万亿美元 ， 比上年 增加 百分之二十五 点五 。因而 ， 日本 去年 海外 纯 资产额 为 ２ ９ ３ ２ · １ ５ 亿美元 ， 连续 ５ 年 居 世界 首位 。 但是 ， 由于 外债 增加 ， 比 上年增长速度 仅为 百分之零点 五 。 而 １ ９ ８ ８ 年 比 １ ９ ８ ７ 年的 增长率 达 百分之二 十一点二 。 （ 完 ）'

In [4]:
# build train and test datasets
text_text = dataset['text'].values
category = dataset['category'].values

train_text = text_text[:2000]
train_category = category[:2000]

test_text = text_text[2000:]
test_category = category[2000:]

In [5]:
import contractions
from bs4 import BeautifulSoup
import unicodedata
import re
import tqdm
import numpy as np
from stop_words import get_stop_words

import contractions
from bs4 import BeautifulSoup
import numpy as np
import re
import tqdm
import unicodedata
import gensim

In [6]:
def stripHTMLtags(text) : 
    soup  = BeautifulSoup(text,"html.parser")
    stripped_text = soup.get_text()
    stripped_text = re.sub(r'[\r|\n\|\r\n]+','\n',stripped_text)
    return stripped_text

def strip_email(text):
    text = re.sub("[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+","",text)
    return text
        
def strip_digits(text):
    text = " ".join(word for word in text.split() if not any(character.isdigit() for character in word))
    return text

def strip_html_tags(text):
  soup = BeautifulSoup(text, "html.parser")
  [s.extract() for s in soup(['iframe', 'script'])]
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+', '\n', stripped_text)
  return stripped_text

def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
  return text

def strip_common_words(text,stemming = False) :
    stop_words = get_stop_words("en")
    if stemming :
        text = [word for word in text.split() if word not in stem(stop_words)]
    else :
        text = [word for word in text.split() if word not in stop_words]
    return text

def getChinese(context):
    filterate = re.compile(u'[^u4E00-\u9FFF ]')
    context = filterate.sub(r'',context)
    return context



In [12]:
from zhon.hanzi import punctuation
from zhon.hanzi import non_stops
from zhon.hanzi import stops
import advertools as adv
stop_words = sorted(adv.stopwords['chinese'])

def pre_process_corpus(docs, stemming = False, rm_common = False):
  norm_docs = []
  for doc in tqdm.tqdm(docs):
    doc = doc.translate(doc.maketrans("\n\t\r", "   "))
    for symbol in string.punctuation:
        doc = doc.replace(symbol," ")
    doc = strip_digits(doc)
    #lower case and remove special characters\whitespaces
    doc = re.sub(' +', ' ', doc)
    doc = doc.strip()  
    doc = re.sub('[a-zA-Z]', '', doc) 
    doc = getChinese(doc)
    doc = re.sub(r"[%s]+" %punctuation,"",doc)
    doc = re.sub(r"[%s]+" %non_stops,"",doc)
    doc = re.sub(r"[%s]+" %stops,"",doc)
    text = gensim.utils.to_unicode(doc).split()
    text = [word.lower() if not word.isupper() else word for word in text]
    text = [t for t in text if t not in stop_words]
    """
    if stemming :
        text = [stem(word) for word in text]
        
    if rm_common :
        text = strip_common_words(text)
    """
    norm_docs.append(" ".join(text))
  
  return norm_docs

In [13]:
norm_train_text = pre_process_corpus(train_text)
norm_test_text = pre_process_corpus(test_text)

100%|██████████| 500/500 [00:13<00:00, 36.81it/s]


In [14]:
norm_train_text[1]

'日本 去年 海外 资产 负债额 创 历史纪录新华社 东京 月 日电 日本 内阁 发表 一份 报告书 去年 日本 海外 资产额 外债 额均 创 历史 纪录 去年 日本 海外 资产额 已达 万亿美元 年 增加 百分之二十点 海外 资产 中 增加 最快 是民间 部门 投资 制造业 日本 购买外国 不动产 在外 办厂 资产额 年 增加 百分 三十九点 达 亿美元 去年 日本 对外 负债额 万亿美元 比上年 增加 百分之二十五 点五 日本 去年 海外 资产额 亿美元 连续 年 居 世界 首位 外债 增加 上年增长速度 仅为 百分之零点 年 年的 增长率 达 百分之二 十一点二 完'

In [16]:
train_text[1]

'\ufeff                       日本 去年 海外 资产 和 负债额 均 创 历史纪录新华社 东京 ５ 月 ２ ５ 日电 日本 内阁 今天 发表 的 一份 报告书 表明 ， 去年 日本 海外 资产额 和 外债 额均 创 历史 最高 纪录 。去年 日本 海外 资产额 已达 １ · ７ ７ １ 万亿美元 ， 比 １９ ８ ８ 年 增加 百分之二十点 五 。 海外 资产 中 增加 最快 的 是民间 部门 的 直接 投资 。 在 制造业 方面 ， 日本 由于 大量 购买外国 的 不动产 和 在外 办厂 等 ， 资产额 比 １ ９ ８ ８ 年 增加 百分 之 三十九点 三 ， 达 １ ５ ４ ３ 亿美元 。去年 日本 对外 负债额 为 １ · ４ ７ ７ ７ 万亿美元 ， 比上年 增加 百分之二十五 点五 。因而 ， 日本 去年 海外 纯 资产额 为 ２ ９ ３ ２ · １ ５ 亿美元 ， 连续 ５ 年 居 世界 首位 。 但是 ， 由于 外债 增加 ， 比 上年增长速度 仅为 百分之零点 五 。 而 １ ９ ８ ８ 年 比 １ ９ ８ ７ 年的 增长率 达 百分之二 十一点二 。 （ 完 ）'

In [17]:
import tensorflow as tf

t = tf.keras.preprocessing.text.Tokenizer()
# fit the tokenizer on the documents
t.fit_on_texts(norm_train_text)
t.word_index['<PAD>'] = 0

In [18]:
train_sequences = t.texts_to_sequences(norm_train_text)
test_sequences = t.texts_to_sequences(norm_test_text)


In [19]:
print("Vocabulary size={}".format(len(t.word_index)))
print("Number of Documents={}".format(t.document_count))

Vocabulary size=75359
Number of Documents=2000


In [20]:
MAX_SEQUENCE_LENGTH = 500

In [21]:
# pad dataset to a maximum review length in words
X_train = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_test = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
X_train.shape, X_test.shape

((2000, 500), (500, 500))

In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
num_classes=10
y_train = le.fit_transform(train_category)
y_test = le.transform(test_category)

In [23]:
nb_classes = 10
y_train = np.eye(nb_classes)[y_train]
y_test = np.eye(nb_classes)[y_test]

In [24]:
"""
print(tf.__version__)
from sklearn.preprocessing import MultiLabelBinarizer
multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit_transform(y_train)
y = multilabel_binarizer.classes_
y
"""

'\nprint(tf.__version__)\nfrom sklearn.preprocessing import MultiLabelBinarizer\nmultilabel_binarizer = MultiLabelBinarizer()\nmultilabel_binarizer.fit_transform(y_train)\ny = multilabel_binarizer.classes_\ny\n'

In [25]:
"""from sklearn.preprocessing import MultiLabelBinarizer
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)
"""

'from sklearn.preprocessing import MultiLabelBinarizer\ny_train = np_utils.to_categorical(y_train)\ny_test = np_utils.to_categorical(y_test)\n'

In [26]:
VOCAB_SIZE = len(t.word_index)

In [28]:
EMBEDDING_DIM = 300 # dimension for dense embeddings for each token
LSTM_DIM = 128 # total LSTM units

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH))
model.add(tf.keras.layers.Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(tf.keras.layers.MaxPooling1D(pool_size = 2))
model.add(tf.keras.layers.LSTM(units = 20, dropout = 0.5, recurrent_dropout = 0.5))
model.add(tf.keras.layers.Dense(units = 10, activation = "softmax"))
model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["categorical_accuracy"])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 300)          22607700  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 128)          115328    
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 250, 128)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 20)                11920     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                210       
Total params: 22,735,158
Trainable params: 22,735,158
Non-trainable params: 0
_________________________________________________________________
None


In [29]:
EPOCHS=10
BATCH_SIZE=128

# callbacks
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                      patience=2,
                                      verbose=1)

# Fit the model
model.fit(X_train, y_train, 
          validation_split=0.1,
          epochs=EPOCHS, 
          batch_size=BATCH_SIZE,
          callbacks=[es], 
          verbose=1)

Epoch 1/10
15/15 [==============================] - 64s 4s/step - loss: 2.2421 - categorical_accuracy: 0.1983 - val_loss: 2.1667 - val_categorical_accuracy: 0.2100
Epoch 2/10
15/15 [==============================] - 64s 4s/step - loss: 2.0219 - categorical_accuracy: 0.2450 - val_loss: 1.9273 - val_categorical_accuracy: 0.2950
Epoch 3/10
15/15 [==============================] - 64s 4s/step - loss: 1.5888 - categorical_accuracy: 0.4844 - val_loss: 1.5217 - val_categorical_accuracy: 0.4500
Epoch 4/10
15/15 [==============================] - 64s 4s/step - loss: 1.0229 - categorical_accuracy: 0.7656 - val_loss: 1.1678 - val_categorical_accuracy: 0.7000
Epoch 5/10
15/15 [==============================] - 67s 4s/step - loss: 0.5520 - categorical_accuracy: 0.9483 - val_loss: 0.9451 - val_categorical_accuracy: 0.7600
Epoch 6/10
15/15 [==============================] - 71s 5s/step - loss: 0.3189 - categorical_accuracy: 0.9867 - val_loss: 0.8433 - val_categorical_accuracy: 0.7700
Epoch 7/10
15/15

In [ ]:
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))